In [1]:
import pandas as pd
import numpy as np
from TextHandler import *

In [2]:
df= pd.read_csv('data/arabic_categorization_data.csv')
df.drop(df.columns[0], axis=1, inplace=True)
df.dropna(inplace=True)
print(df.head())

                                                text     type
0  \nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...  culture
1  \nتحصل كتاب "المصحف وقراءاته" الذي ألفه باحثون...  culture
2  \nاستنكرت إدارة المسرح الوطني التونسي الحملة ا...  culture
3  \nاحتضن جناح تونس في القرية الدولية للأفلام بم...  culture
4  \nشهدت برلين أمس الجمعة افتتاح مسجد فريد من نو...  culture


In [3]:
data=df['text']
print(data[:1])
preprocessed_data=preprocess_text(data[:1])
print(preprocessed_data)
# for n,text in enumerate(preprocessed_data):
#     for i in  range(1,5):
#         if(len(text.split())<i):
#             break
#         print(i,'-gram:',n_gram(text,i))
#         print(i,'Tf-Idf:',tf_idf(preprocessed_data,i)[n])

#     print('')

0    \nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...
Name: text, dtype: object
['أشرف رئيس الجمهورية الباجي قايد السبسي اليوم بقصر قرطاج موكب منح الوسام الوطني للاستحقاق الثقافي الفنانين والمبدعين بمناسبة انعقاد أيام قرطاج السينمائية والفنانون الصنف عبد الرحمان سيساكو موريتانيا جميل راتب مصر ميشال خليفي فلسطين ادريسا ودراغو بوركينا فاسو محمد ملص سوريا رضا الباهي تونس الحضور عمر الخليفي تونس الصنف الثاني عبد العزيز بن ملوكة تونس نجيب تونس منصف شرف الدين تونس الحضور الصنف الثالث ابراهيم تونس الخياطي تونس الحضور تونس عليها الحضور شوقي الماجري تونس الصنف الرابع كوثر بن تونس']
